47.7 PROJECT. Определяем количество ботов на сайте

Задания

1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.
2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

In [60]:
import json 
import requests
from pprint import pprint
import pandas as pd

In [61]:
# Выгружаем данные

token='AQAAAAAkNoTeAAfLzlxPufiUjk0aqJOFMB7EMYA'
headers = {'Authorization': 'OAuth ' + token}

params = {'metrics': 'ym:s:robotPercentage,ym:s:visits', 
          'dimensions': 'ym:s:isRobot,ym:s:hour,ym:s:lastsignTrafficSource',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',
            'limit': 100000,     
          'ids': 30177909}

response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data = response.json()

In [62]:
# Преобразуем данные

def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)

# Из полученнных данных делаем датафрейм

metrika_df = pd.DataFrame(metrika_list_of_dicts)
metrika_df.columns=['isRobot','Hour','Source','robotPercentage','Visits']
display(metrika_df)

,isRobot,Hour,Source,robotPercentage,Visits
0,Robots,00:00,Direct traffic,100.0,2.0
1,Robots,00:00,Search engine traffic,100.0,17.0
2,Robots,01:00,Direct traffic,100.0,4.0
3,Robots,01:00,Search engine traffic,100.0,8.0
4,Robots,01:00,Cached page traffic,100.0,1.0
...,...,...,...,...,...
177,People,22:00,Social network traffic,0.0,3.0
178,People,23:00,Direct traffic,0.0,19.0
179,People,23:00,Link traffic,0.0,1.0
180,People,23:00,Search engine traffic,0.0,104.0


In [63]:
#  Преобразуем время в соответсвующий формат

metrika_df['Hour'] = pd.to_datetime(metrika_df['Hour'])
metrika_df['Hour'] = metrika_df['Hour'].dt.time
metrika_df

,isRobot,Hour,Source,robotPercentage,Visits
0,Robots,00:00:00,Direct traffic,100.0,2.0
1,Robots,00:00:00,Search engine traffic,100.0,17.0
2,Robots,01:00:00,Direct traffic,100.0,4.0
3,Robots,01:00:00,Search engine traffic,100.0,8.0
4,Robots,01:00:00,Cached page traffic,100.0,1.0
...,...,...,...,...,...
177,People,22:00:00,Social network traffic,0.0,3.0
178,People,23:00:00,Direct traffic,0.0,19.0
179,People,23:00:00,Link traffic,0.0,1.0
180,People,23:00:00,Search engine traffic,0.0,104.0


In [64]:
# Посчитаем количество ботов заходивших на сайт и их долю от общего трафика
# Создаем сводную таблицу с количеством визитов людей и роботов
pt_robot_visits = metrika_df.pivot_table(values = ['Visits'], columns = ['isRobot'], aggfunc = 'sum')
# Считаем долю роботов
pt_robot_visits['robot_visits%'] = round(pt_robot_visits['Robots'] * 100 / (pt_robot_visits['Robots'] + pt_robot_visits['People']), 2)
print(pt_robot_visits)

isRobot  People  Robots  robot_visits%
Visits   3353.0   962.0          22.29


Из таблицы выше можно сделать вывод, что за май 2020 года на наш сайт зашло 962 робота, что составило 22.29% от всех посетителей

In [65]:
# Выведем сводную таблицу с разбивкой прихода роботов по часам и источникам

pt_robot_source_time = metrika_df.loc[metrika_df['isRobot'].isin(['Robots'])].pivot_table(values = ['Visits'],
                                              index = ['Hour'],
                                              columns = ['Source'],
                                              aggfunc = 'sum',
                                              fill_value = 0,
                                              margins = True)
print(pt_robot_source_time)


                      Visits                                               \
Source   Cached page traffic Direct traffic Internal traffic Link traffic   
Hour                                                                        
00:00:00                   0              2                0            0   
01:00:00                   1              4                0            0   
02:00:00                   0              2                0            0   
03:00:00                   0              2                0            1   
04:00:00                   0              3                0            0   
05:00:00                   0              3                0            0   
06:00:00                   0              3                0            2   
07:00:00                   0              5                0            1   
08:00:00                   0              5                0            1   
09:00:00                   0             12                1            1   

Как видно из таблицы выше больше всего роботов (739) пришло из источника Search engine traffic. Наибольшая активность роботов была в дневное время с 11 до 17 часов.